Consolidate the folds into a single set of results.

In particular, compute the mean and standard deviations of the models in the accuracy, recall and F-score. 

In [1]:
import os
from glob import glob

# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf
import pandas as pd
import numpy as np

from ScalableLib.mixture.layers import create_models
from ScalableLib.classifier import Multiband as multiband

2024-10-02 15:35:49.711131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 15:35:49.711172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 15:35:49.712459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 15:35:51.089614: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# To see if the system recognises the GPU
device = 1
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

device_name = tf.config.experimental.get_device_details(devices[device])['device_name']
print("Using {}".format(device_name))

Using NVIDIA GeForce RTX 2080 Ti


Read the Results folder.

In [3]:
survey = 'ZTF'
path = os.path.join('../../02_CreateRecords/', survey, 'Folds/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../../02_CreateRecords/ZTF/Folds/Fold_1',
 '../../02_CreateRecords/ZTF/Folds/Fold_2',
 '../../02_CreateRecords/ZTF/Folds/Fold_3',
 '../../02_CreateRecords/ZTF/Folds/Fold_4',
 '../../02_CreateRecords/ZTF/Folds/Fold_5',
 '../../02_CreateRecords/ZTF/Folds/Fold_6',
 '../../02_CreateRecords/ZTF/Folds/Fold_7']

In [4]:
if not os.path.exists('./Results/'):
    os.mkdir('./Results/')

For each fold, read the models and evaluate on the test set. Save the results

In [5]:
train_args = {
            'hidden_size_bands':[128,128, 128],
            'hidden_size_central':[128, 128],
            'fc_layers_bands':[128,128,128],
            'fc_layers_central':[128,128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':128,
            'dropout':0.30,
            'lr':[[5e-3]*2, 2.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':True,# Not Used as intended, for initialization
            'mode' : 'classifier'
            }
loss_weights = {'Class':1.0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': [],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,  
                    'N_skip': 3, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': False,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



In [6]:
sauces = []
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../../02_CreateRecords/'+survey+'/', './').replace('/Folds/', '/Results/')
    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    
        
    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = base_dir+'metadata_preprocess.json'  
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers.pkl')
    # Define the train args
    train_args = {**train_args, **train_args_specific}
    
    train_files = base_dir+'train/*.tfrecord'
    val_files = base_dir+'val/*.tfrecord'
    test_files = base_dir+'test/*.tfrecord'
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    # Create the Mixture of Experts model, from the original multiband model
    model_base = create_models(new)
    model = model_base.creat_split_models(train_args)

    # Define the early stopping callback
    es = tf.keras.callbacks.EarlyStopping(monitor='accuracy', **callbacks_args)
    # Fit the model to the original data
    model.fit(new.dataset_train, validation_data=new.dataset_val, epochs=300,callbacks = [es] )

    # Store the alpha coefficients per fold
    sauces_ = {i.name:i for i in model.layers if 'Sauce_' in i.name}
    scales = {key:tf.nn.softmax(sauces_[key].scale).numpy() for key in sauces_.keys()}
    sauces.append(scales)
    # Test
    dfs = []
    for batch in new.dataset_test:
        prediction = model(batch[0])
        y_pred = prediction['Class'].numpy().argmax(axis=1)
        y_pred = [new.trans[i] for i in y_pred]
        ID = batch[0]['ID'].numpy()

        y_true = batch[1]['Class'].numpy().argmax(axis=1)
        y_true = [new.trans[i] for i in y_true]

        df = pd.DataFrame(np.array([ID, y_pred, y_true]).transpose(), columns=['ID', 'Class', 'Pred'])
        df.ID = df.ID.str.decode('UTF-8')
        dfs.append(df)
    dfs = pd.concat(dfs, axis=0)
    dfs.to_csv(path_results_fold+'/Classification_test.dat', index=False, index_label=False)
    
pd.DataFrame(sauces).to_csv(path_results_fold+'/sauces.dat', index=False, index_label=False)    

./Results/Fold_1/Models/20241002-1535
Epoch 1/300


I0000 00:00:1727897808.125079   98190 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


241/241 [==============================] - 74s 176ms/step - loss: 0.9222 - accuracy: 0.6984 - val_loss: 0.5945 - val_accuracy: 0.8014
Epoch 2/300
241/241 [==============================] - 37s 151ms/step - loss: 0.6149 - accuracy: 0.7965 - val_loss: 0.5277 - val_accuracy: 0.8209
Epoch 3/300
241/241 [==============================] - 35s 145ms/step - loss: 0.5280 - accuracy: 0.8223 - val_loss: 0.5110 - val_accuracy: 0.8250
Epoch 4/300
241/241 [==============================] - 35s 144ms/step - loss: 0.4931 - accuracy: 0.8320 - val_loss: 0.4582 - val_accuracy: 0.8417
Epoch 5/300
241/241 [==============================] - 35s 144ms/step - loss: 0.4513 - accuracy: 0.8463 - val_loss: 0.4139 - val_accuracy: 0.8556
Epoch 6/300
241/241 [==============================] - 35s 146ms/step - loss: 0.4178 - accuracy: 0.8596 - val_loss: 0.3930 - val_accuracy: 0.8649
Epoch 7/300
241/241 [==============================] - 35s 146ms/step - loss: 0.3872 - accuracy: 0.8690 - val_loss: 0.3669 - val_accurac

In [7]:
1

1